# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf, isnan, when, sort_array, asc, lag, floor
from pyspark.sql import Window
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import sum as Fsum

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import DoubleType

import datetime
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# create a Spark session

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [7]:
from pyspark.sql import SparkSession
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-9b9eab8f-0bde-4b92-b104-d34aeab1cdf6',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'JgcZpYUhTBgq4dpgZEGYt8m88AODATROOEbdAQv16rpt'
}

configuration_name = 'os_21c0332572af4b98b556fb9e269421c9_configs'
cos = ibmos2spark.CloudObjectStorage(
    sc, credentials, configuration_name, 'bluemix_cos')

spark = SparkSession.builder.getOrCreate()

df_data_1 = spark.read.json(cos.url(
    'medium-sparkify-event-data.json', 'udacityconsponepoject-donotdelete-pr-ttedboic8u3ybu'))
df_data_1.take(5)

ModuleNotFoundError: No module named 'ibmos2spark'

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [6]:
df_spark = df_data_1
df_spark.count()

NameError: name 'df_data_1' is not defined

In [ ]:
df_spark = df_spark.filter(df_spark['userId'] != "")
df_spark.count()

In [ ]:
# df_spark.show()

In [ ]:
df_spark.select('userId').dropDuplicates().count()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
def df_Value_Summary(df, col_watch, col_value, nhead=5):
    '''
    Function: show the freq and the most nhead propotion value of col,and show the value and count of Null 
    Input:
        df: the value count of col dataFrame
        col_watch:the watch column
        col_value:the value which to be watch by col_watch
        nhead:how many most propotion value to show 
    Output:DataFrame including the freq and the most nhead propotion value of col,and show the value and count of Null 

    排序还是有些问题，注意检查
    '''
    tmp = df.select(col_value, col_watch).dropDuplicates().groupby(col_watch).count().toPandas(
    ).rename_axis({'count': col_value}, axis=1).sort_values(by=col_value, ascending=False)

    value = pd.DataFrame(['value {}'.format(i) for i in range(nhead)], index=range(
        nhead)).join(tmp.iloc[:, 0], how='left').set_index(0).T

    freq = pd.DataFrame(['freq {}'.format(i) for i in range(nhead)], index=range(
        nhead)).join(tmp.iloc[:, 1], how='left').set_index(0).T

    freq.index = [col_watch]

    null_count = tmp[tmp[col_watch] != tmp[col_watch]].shape[0]

    if null_count == 0:
        nnull = 0
    else:
        nnull = tmp[tmp[col_watch] != tmp[col_watch]
                    ].reset_index().loc[0, col_value]

    freqother = pd.DataFrame([nnull, tmp[col_value].sum() - nnull - freq.sum(
        axis=1).sum()], index=['freqNull', 'freqOther']).T.rename_axis({0: col_watch})

    freqother.index = [col_watch]

    count_value = pd.DataFrame({'count': [tmp.shape[0]]}, index=[col_watch])

    freq_sum = pd.DataFrame(
        {'sum': [np.sum(tmp[col_value])]}, index=[col_watch])

    freq_min = pd.DataFrame(
        {'min': [np.min(tmp[col_value])]}, index=[col_watch])

    freq_max = pd.DataFrame(
        {'max': [np.max(tmp[col_value])]}, index=[col_watch])

    freq_mean = pd.DataFrame(
        {'mean': [np.mean(tmp[col_value])]}, index=[col_watch])

    freq_median = pd.DataFrame(
        {'median': [np.median(tmp[col_value])]}, index=[col_watch])

    op = pd.concat([count_value, freq_sum, freq_min, freq_max,
                   freq_mean, freq_median, value, freq, freqother], axis=1)

    return op

In [ ]:
col_watch_list = ['artist', 'auth', 'firstName', 'gender',
                  'level', 'location', 'page', 'song', 'status', 'userAgent']
col_value = 'userId'
df_watch_All = pd.DataFrame()
for col_watch in col_watch_list:
    df_watch_summary = df_Value_Summary(df_spark, col_watch, col_value, 5)
    df_watch_All = pd.concat([df_watch_All, df_watch_summary], axis=0)

df_watch_All

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.